In [1]:
# %%
# Step 1: Import libraries
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from dotenv import load_dotenv

# %%
# Step 2: Load environment variables
load_dotenv()

pg_user = os.environ['PG_USER']
pg_password = os.environ['PG_PASSWORD']
pg_host = os.environ['PG_HOST']
pg_port = os.environ['PG_PORT']
pg_db = os.environ['PG_DB']

# %%

In [2]:
# Step 3: Create engine
from sqlalchemy import create_engine

engine = create_engine(
    f'postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_db}'
)


## Diagnostic

### Business Question
What are the most common URL feature combinations (HTTPS usage and IP address presence) among verified phishing URLs?

In [4]:
query1 = '''
WITH length_buckets AS (
    SELECT 
        CASE 
            WHEN u.url_length < 30 THEN 'Short'
            WHEN u.url_length BETWEEN 30 AND 60 THEN 'Medium'
            ELSE 'Long'
        END AS length_category,
        f.phishing_status,
        COUNT(*) AS url_count
    FROM raw.fact_phishing_urls f
    JOIN raw.dim_url_features u
        ON f.feature_id = u.feature_id
    GROUP BY length_category, f.phishing_status
),
ranked_lengths AS (
    SELECT *,
           RANK() OVER (PARTITION BY phishing_status ORDER BY url_count DESC) AS rank_within_status
    FROM length_buckets
)
SELECT *
FROM ranked_lengths
WHERE phishing_status = 'Phishing (Verified)'
ORDER BY url_count DESC;

'''

df_q1 = pd.read_sql(query1, con=engine)
df_q1


,length_category,phishing_status,url_count,rank_within_status
0,Medium,Phishing (Verified),47,1
1,Short,Phishing (Verified),10,2


### Insight  
Among verified phishing URLs, the **"Medium" length category (30–60 characters)** is the most common, with 47 instances, compared to only 10 in the "Short" category. No long URLs appeared in this filtered result set, suggesting that phishers tend to use medium-length URLs when crafting deceptive links.

### Recommendation  
Awareness campaigns and security tools should pay closer attention to medium-length URLs that appear suspicious or unfamiliar. URL length, while not a definitive indicator of phishing, can be used as a risk signal when combined with other red flags like lack of HTTPS or suspicious domains.

### Prediction  
If the trend continues, phishing detection models that factor in **medium-length URLs** as a high-risk feature may improve early identification of phishing attacks, especially when integrated with additional metadata like domain age or SSL usage.


## Descriptive

### Business Question
Which URL length categories are most frequently associated with verified phishing URLs, and how do they rank?

In [3]:
query1 = '''

WITH feature_summary AS (
    SELECT 
        f.phishing_status,
        u.uses_https,
        u.has_ip_address,
        COUNT(*) AS url_count
    FROM raw.fact_phishing_urls f
    JOIN raw.dim_url_features u
        ON f.feature_id = u.feature_id
    GROUP BY f.phishing_status, u.uses_https, u.has_ip_address
),
ranked_summary AS (
    SELECT *,
        RANK() OVER (
            PARTITION BY phishing_status 
            ORDER BY url_count DESC
        ) AS rank_within_status
    FROM feature_summary
)
SELECT * 
FROM ranked_summary
WHERE rank_within_status <= 3;

'''

df_q1 = pd.read_sql(query1, con=engine)
df_q1


,phishing_status,uses_https,has_ip_address,url_count,rank_within_status
0,Phishing (Verified),True,False,57,1


### Insight
The most common feature combination among verified phishing URLs (57 instances) includes the use of HTTPS and the absence of an IP address. This indicates that phishing websites often adopt HTTPS to appear more legitimate and trustworthy. Also, the most common verified phishing URLs (57 instances) do **not** contain an IP address. This suggests that attackers are favoring domain-based URLs over raw IP addresses, likely to make the links appear more trustworthy and avoid detection.

### Recommendation
Security awareness training should emphasize that HTTPS does not guarantee a site's legitimacy. Detection tools and user training should not rely solely on spotting IP addresses in URLs as phishing indicators.

### Prediction
As phishing techniques evolve, we can expect an increasing number of phishing sites to use HTTPS by default, reducing the reliability of HTTPS as a standalone trust signal. The trend of using domain-based phishing URLs will continue to rise, as attackers mimic legitimate domain structures to evade user suspicion and automated filters.
